In [1]:
import numpy
from langchain.llms import GooglePalm
import numpy
import os
from dotenv import load_dotenv
load_dotenv() 

api_key = os.environ["GOOGLE_API_KEY"]

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

In [2]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine

In [3]:
db_driver = "ODBC Driver 17 for SQL Server"
db_server = "NAVEENTAMIZHAN"
db_name = "nav_electric_vehicles"
Database_Connect = f"mssql://@{db_server}/{db_name}?Driver={db_driver}"

engine = create_engine(Database_Connect)
db = SQLDatabase(engine, sample_rows_in_table_info=3)

print(db.table_info)

C:\Users\Naveen\AppData\Local\Temp\ipykernel_15248\2588683582.py:6: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  engine = create_engine(Database_Connect)



CREATE TABLE [TestTable] (
	[IDName] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NULL
)

/*
3 rows from TestTable table:
IDName
Nir10ma0la1
1A0ru0na2
S1h00ashi3
*/


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL IDENTITY(1,1), 
	ev_car_id SMALLINT NOT NULL, 
	percentage_discount DECIMAL(5, 2) NULL, 
	CONSTRAINT [PK__discount__BDBE9EF9722F0DC0] PRIMARY KEY (discount_id), 
	CONSTRAINT [FK__discounts__ev_ca__7B5B524B] FOREIGN KEY(ev_car_id) REFERENCES ev_cars (ev_car_id)
)

/*
3 rows from discounts table:
discount_id	ev_car_id	percentage_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE ev_cars (
	ev_car_id SMALLINT NOT NULL, 
	brand NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	color NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	model NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	price INTEGER NOT NULL, 
	stock TINYINT NOT NULL, 
	CONSTRAINT [PK_ev_cars] PRIMARY KEY (ev_car_id)
)

/*
3 rows from ev_cars table:
ev_car_

In [4]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many tesla do we have left for tesla in premium model and white color?")



> Entering new SQLDatabaseChain chain...
How many tesla do we have left for tesla in premium model and white color?
SQLQuery:SELECT stock FROM ev_cars WHERE brand = 'Tesla' AND model = 'premium' AND color = 'white'
SQLResult: 
Answer:155
> Finished chain.


In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many white color and green color Tesla's car I have?")



> Entering new SQLDatabaseChain chain...
How many white color and green color Tesla's car I have?
SQLQuery:SELECT COUNT(*) FROM ev_cars WHERE brand = 'Tesla' AND (color = 'Green' OR color = 'White')
SQLResult: [(68,)]
Answer:68
> Finished chain.


In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many ev_cars do we have left for Tata in premium_customized model and white color?")



> Entering new SQLDatabaseChain chain...
How many ev_cars do we have left for Tata in premium_customized model and white color?
SQLQuery:SELECT stock FROM ev_cars WHERE brand = 'Tata' AND model = 'premium_customized' AND color = 'white'
SQLResult: 
Answer:100
> Finished chain.


In [6]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How much is the total price of the inventory for all base-model evcars?")



> Entering new SQLDatabaseChain chain...
How much is the total price of the inventory for all base-model evcars?
SQLQuery:SELECT SUM(price) FROM ev_cars WHERE model = 'base'
SQLResult: [(266856509,)]
Answer:266856509
> Finished chain.


In [7]:
qns2 = db_chain.run("""SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.percentage_discount, 0)) / 100)) AS total_revenue
FROM (
    SELECT SUM(price * stock) AS total_amount, ev_car_id
    FROM ev_cars
    WHERE brand = 'Tesla'
    GROUP BY ev_car_id
) a
LEFT JOIN discounts ON a.ev_car_id = discounts.ev_car_id;

""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.percentage_discount, 0)) / 100)) AS total_revenue
FROM (
    SELECT SUM(price * stock) AS total_amount, ev_car_id
    FROM ev_cars
    WHERE brand = 'Tesla'
    GROUP BY ev_car_id
) a
LEFT JOIN discounts ON a.ev_car_id = discounts.ev_car_id;


SQLQuery:SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.percentage_discount, 0)) / 100)) AS total_revenue
FROM (
    SELECT SUM(price * stock) AS total_amount, ev_car_id
    FROM ev_cars
    WHERE brand = 'Tesla'
    GROUP BY ev_car_id
) a
LEFT JOIN discounts ON a.ev_car_id = discounts.ev_car_id;
SQLResult: [(Decimal('31959014534.500000'),)]
Answer:31959014534.5
> Finished chain.


In [8]:
qns5 = db_chain.run("""SELECT SUM(CAST(price AS BIGINT) * CAST(stock AS BIGINT)) AS total_price
FROM Cars
WHERE brand = 'Mahindra';
""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(CAST(price AS BIGINT) * CAST(stock AS BIGINT)) AS total_price
FROM Cars
WHERE brand = 'Mahindra';

SQLQuery:SELECT SUM(CAST(price AS BIGINT) * CAST(stock AS BIGINT)) AS total_price
FROM ev_cars
WHERE brand = 'Mahindra';
SQLResult: [(30126787806,)]
Answer:30126787806
> Finished chain.


### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [8]:
few_shots = [
    {'Question' : "How many ev_cars do we have left for Tata in premium_customized model and white color?",
     'SQLQuery' : "SELECT stock FROM ev_cars WHERE brand = 'Tata' AND color = 'White' AND model = 'premium_customized'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "100"},
    {'Question' : "How many tesla do we have left for tesla in premium model and white color?",
     'SQLQuery' : "SELECT stock FROM ev_cars WHERE brand = 'Tesla' AND model = 'premium' AND color = 'white'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "155"},
    {'Question': "How much is the total price of the inventory for all base-model evcars?",
     'SQLQuery':"SELECT SUM(price) FROM ev_cars WHERE model = 'base'",
     'SQLResult': "Result of the SQL query",
     'Answer': "266856509"},
    {'Question': "If we have to sell all the Tesla ev_cars today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.percentage_discount, 0)) / 100)) AS total_revenue
FROM (
    SELECT SUM(price * stock) AS total_amount, ev_car_id
    FROM ev_cars
    WHERE brand = 'Tesla'
    GROUP BY ev_car_id
) a
LEFT JOIN discounts ON a.ev_car_id = discounts.ev_car_id;
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Tesla Car today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock) FROM Cars WHERE brand = 'Tesla",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Tesla's shirt I have?",
     'SQLQuery' : "SELECT sum(stock) FROM ev_cars WHERE brand = 'Tesla' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "how much sales amount will be generated if we sell all premium electric cars today in Tata brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.percentage_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, ev_car_id from ev_cars where brand = 'Tata' and model="premium"
group by ev_car_id) a left join discounts on a.ev_car_id = discounts.ev_car_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]


In [9]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

D:\conda\ml_project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
to_vectorize

["How many ev_cars do we have left for Tata in premium_customized model and white color? SELECT stock FROM ev_cars WHERE brand = 'Tata' AND color = 'White' AND model = 'premium_customized' Result of the SQL query 100",
 "How many tesla do we have left for tesla in premium model and white color? SELECT stock FROM ev_cars WHERE brand = 'Tesla' AND model = 'premium' AND color = 'white' Result of the SQL query 155",
 "How much is the total price of the inventory for all base-model evcars? SELECT SUM(price) FROM ev_cars WHERE model = 'base' Result of the SQL query 266856509",
 "If we have to sell all the Tesla ev_cars today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.percentage_discount, 0)) / 100)) AS total_revenue\nFROM (\n    SELECT SUM(price * stock) AS total_amount, ev_car_id\n    FROM ev_cars\n    WHERE brand = 'Tesla'\n    GROUP BY ev_car_id\n) a\nLEFT JOIN discounts ON a.ev_car_id = discou

In [11]:
vectorstore = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)